<a href="https://colab.research.google.com/github/MikeSalnikov/Data-analysis-algorithms/blob/main/HW3_web3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 3

1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - использовать np.clip или np.where). 
2. На данных из урока изучите влияние гиперпараметров на ошибку алгоритма. 
3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса "1". На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model. 
4. Создайте функцию calc_pred, возвращающую предсказанные классы (0 или 1). На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, а также порог вероятности. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [3]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

In [4]:
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

In [5]:
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - использовать np.clip или np.where).

In [ ]:
# def calc_logloss(y, y_pred):
#     err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
#     return err

In [6]:
# приму, что 0,9999999999 близко к единице, а 0,0000000001 близко к нулю:  

def calc_logloss(y, y_pred):
    y_pred = np.where(y_pred==1, y_pred-1e-10, np.where(y_pred==0, y_pred+1e-10, y_pred))
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [7]:
# пример применения: 

y1 = np.array([1, 0])
y_pred1 = np.array([0.8, 0.1])
calc_logloss(y1, y_pred1)

0.164252033486018

In [8]:
# теперь неплохой пример применения: 

y2 = np.array([1, 0])
y_pred2 = np.array([1, 0.2])
calc_logloss(y2, y_pred2)

0.11157177570710486

In [9]:
# теперь неплохой пример применения: 

y3 = np.array([1, 0])
y_pred3 = np.array([0, 0.5])
calc_logloss(y3, y_pred3)

11.8594990552502

In [ ]:
2. На данных из урока изучите влияние гиперпараметров на ошибку алгоритма. 

In [10]:
def eval_model(X, y, iterations, eta):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [11]:
W = eval_model(X_st, y, 500, 1e-4)

# количество итераций 500, eta = 0.0001 - ошибка 1.067

0 [ 0.49667621 -0.13840939  0.6476858   1.52297324] 1.1785958344356262
50 [ 0.494784   -0.14564801  0.6475462   1.52014828] 1.1657985749255426
100 [ 0.49290109 -0.15285535  0.64740132  1.51733474] 1.1531112685708473
150 [ 0.49102761 -0.16003088  0.64725118  1.51453281] 1.140535275330502
200 [ 0.48916364 -0.16717404  0.64709581  1.51174267] 1.1280719326917483
250 [ 0.48730929 -0.17428428  0.64693524  1.50896452] 1.1157225565960736
300 [ 0.48546465 -0.18136107  0.64676951  1.50619853] 1.1034884426224387
350 [ 0.48362982 -0.18840385  0.64659868  1.5034449 ] 1.0913708674192037
400 [ 0.48180488 -0.19541206  0.64642281  1.50070383] 1.0793710903721336
450 [ 0.47998993 -0.20238516  0.64624195  1.49797551] 1.0674903554915993


In [13]:
W = eval_model(X_st, y, 1000, 1e-3)

# количество итераций 1000, eta = 0.001 - ошибка 0.522

0 [ 0.49633477 -0.13971518  0.64766116  1.52246371] 1.1785958344356262
100 [ 0.46038628 -0.27784345  0.64391879  1.46838117] 0.9449083796207038
200 [ 0.42880798 -0.398758    0.63895621  1.42057357] 0.76478635339061
300 [ 0.40211124 -0.49592634  0.63495855  1.38143191] 0.6465819250979653
400 [ 0.3802473  -0.56578851  0.63387547  1.35226238] 0.5834626177311397
500 [ 0.36227303 -0.61296537  0.6357139   1.33157192] 0.5531388819143023
600 [ 0.34705426 -0.6447563   0.63955258  1.31689489] 0.5381879835818275
700 [ 0.33375348 -0.66658248  0.64461446  1.30632368] 0.5301812678912075
800 [ 0.32182152 -0.6818318   0.65041019  1.29860717] 0.5254538963664757
900 [ 0.31089651 -0.69260819  0.65664232  1.29292959] 0.5223650188440195


In [14]:
W = eval_model(X_st, y, 5000, 1e-2)

# количество итераций 5000, eta = 0.01 - ошибка 0.424

0 [ 0.49292028 -0.15277306  0.64741473  1.51736839] 1.1785958344356262
500 [ 0.03982235 -0.72064774  0.8585723   1.32719824] 0.49098749514946327
1000 [-0.16731955 -0.75357199  0.95172595  1.4635263 ] 0.4764623107233352
1500 [-0.33588235 -0.79003576  0.98228783  1.60994271] 0.4660058881458373
2000 [-0.49197    -0.82105502  0.98635336  1.74725905] 0.4571548777302
2500 [-0.64217604 -0.84600899  0.97571427  1.87329246] 0.4493121111978352
3000 [-0.78827285 -0.86584359  0.95540388  1.98934071] 0.4421846628745265
3500 [-0.93081573 -0.88160297  0.92822259  2.09708433] 0.43559915549768197
4000 [-1.07003706 -0.89415928  0.89602422  2.19797368] 0.42944618066185863
4500 [-1.20607612 -0.90420079  0.86014982  2.29316907] 0.42365350540190166


In [15]:
W = eval_model(X_st, y, 10000, 1e-1)

# количество итераций 10000, eta = 0.1 - ошибка 0.238

0 [ 0.45877546 -0.2833519   0.6449505   1.46641523] 1.1785958344356262
1000 [-2.52102182 -0.94535892  0.39937552  3.13420485] 0.37533367618380076
2000 [-4.27828712 -0.98168834 -0.27476935  4.26319069] 0.3266312200993096
3000 [-5.56189789 -1.0318158  -0.73874052  5.15242458] 0.29991853172016747
4000 [-6.58645538 -1.0857742  -1.0912958   5.89903883] 0.28252262722907645
5000 [-7.45579588 -1.13940477 -1.38052844  6.55325919] 0.26979795435357545
6000 [-8.22400342 -1.19158043 -1.63004704  7.14381634] 0.2597485783476198
7000 [-8.92162316 -1.24206841 -1.8526335   7.6879962 ] 0.25139381160087004
8000 [-9.56712984 -1.29090081 -2.05579238  8.19673599] 0.24419878346796203
9000 [-10.17234391  -1.33818162  -2.24422182   8.677278  ] 0.23784703122543333


Вывод: при увеличении количества итераций и скорости (множителя) eta ошибка уменьшается. Правда eta нужно увеличивать аккуратно, чтобы не проскочить минимум при пошаговом градиентном спуске. 

3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса "1". На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model. 

In [16]:
def calc_pred_proba(X, W):
    y_pred_proba = sigmoid(np.dot(X, W))
    return y_pred_proba

In [17]:
calc_pred_proba(X_st, W)

array([0.34909794, 0.16499718, 0.99721038, 0.00128184, 0.73112245,
       0.32370941, 0.99918939, 0.0911856 , 0.40772414, 0.99034837])

4. Создайте функцию calc_pred, возвращающую предсказанные классы (0 или 1). На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, а также порог вероятности.

In [18]:
def calc_pred(X, W, threshold):
    m = X.shape[0]
    y_predicted = np.zeros(m)
    A = np.squeeze(sigmoid(np.dot(X, W)))
    for i in range(A.shape[0]):
        if (A[i] > threshold): 
            y_predicted[i] = 1
        elif (A[i] <= threshold):
            y_predicted[i] = 0
    return y_predicted

In [19]:
calc_pred(X_st, W, 0.5)

array([0., 0., 1., 0., 1., 0., 1., 0., 0., 1.])